In [1]:
# From the API calls the flow on a input row of data to get
# a prediction value from the ML models
# The api_py_funtion is the code that will run on the API
# deployed onto kubernetes.  The rest of the code is stored
# elsewhere
import api.dispatcher_functions as dispatcher

In [ ]:
# Establish connection
def load_project(projectKey,apiKey):
    dataiku.set_remote_dss("url", apiKey)
    client = dataiku.api_client()
    
    project = client.get_project(projectKey)
    
    project_string = project.project_key + '.'
    
    jobId=0
    
    return inputDb, project, project_string

In [ ]:
# Get the handles on the data
def getData(inputDb, project, project_string, data):
#     Running Random record dataset
    recipeObj = project.get_recipe('RECIPE NODE NAME HERE')
    recipeObj.run()
    recipeObj = project.get_recipe('RECIPE NODE NAME HERE')
    recipeObj.run()

#     ### START PYTHON COMPUTE RECIPE ###
    debug = 0
    start = time.perf_counter()

    ds = dataiku.Dataset(project_string + inputDb)
    df = ds.get_dataframe()
    
    ds_m = dataiku.Dataset("DATASET NAME HERE")
    mfdf = ds_m.get_dataframe()

    folder = dataiku.Folder("FOLDER NODE NAME HERE")
    folder_info = folder.get_info()

    with folder.get_download_stream("folder.json") as f:
        jsonData = json.load(f)
    load_time = str(time.perf_counter() - start)
    
    return df, jsonData, mfdf

In [ ]:
# Perform clean transformations on data
def data_engineer(df, jsonData, mfdf):
    # doing data engineering
    temp = time.perf_counter()

    final_object = engineer.vectorize(df,jsonData,mfdf)
    select_final = final_object.final_data.copy()
    de_time = str(time.perf_counter() - temp)

    return select_final

In [ ]:
# Run the data through the flow
def inference_pipeline(select_final):
    temp = time.perf_counter()
    cols = list(select_final.columns)
    for i in cols:
        select_final[i] = select_final[i].apply(lambda x: ''.join(str(x)))

    jobId = select_final['job_id'][0]
    prediction_object = inference.Inference_pipeline_batch(select_final, model_dict.model_dict)

    return prediction_object, jobId

In [ ]:
# format the final prediction response
def final_format(prediction_object):
    prediction_final = prediction_object.final_data.copy()
    prediction_final.columns = prediction_final.columns.str.replace(',', '_')

    prediction_final['final_prediction'] = prediction_final['cc_prediction']+ "_" + prediction_final['ac_prediction'] + "_" + prediction_final['fpc'] + ' ' + prediction_final['cc_prediction'] + "_" + prediction_final['ac_prediction'] + "_" + prediction_final['spc'] + ' ' + prediction_final['cc_prediction'] + "_" + prediction_final['ac_prediction'] + "_" + prediction_final['tpc']
    return prediction_final._get_value(0,'final_prediction')

In [ ]:
# Code on API dispatcher in kubernetes
# Calls the above functions in consecutive order
def api_py_function(projectKey, apiKey, data, inputDb):
    project, project_string = load_project(projectKey,apiKey)
    df, jsonData, mfdf = getData(inputDb, project, project_string, data)
    select_final = data_engineer(df, jsonData, mfdf)
    prediction_object, jobId = inference_pipeline(select_final)
    prediction = final_format(prediction_object)
    
    return prediction

In [ ]:
data = 'PLACE DATA PREDICTION ROW HERE'
inputDb = 'DATASET NAME HERE'
projectKey = 'PROJECT KEY HERE'
# Call the dispatcher function
print(api_py_function(projectKey, apiKey, data, inputDb))